In [1]:
import pandas as pd

# load earthquake events from CSV file
data = pd.read_csv('query.csv')

# show header and rows
data.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-02-10T16:16:15.825Z,-20.6864,-175.4586,10.00,4.9,mb,NaN,145.0,6.792,0.74,...,2022-02-10T17:07:13.040Z,"56 km NNW of Nuku‘alofa, Tonga",earthquake,15.8,1.9,0.064,76.0,reviewed,us,us
1,2022-02-10T10:27:32.765Z,-10.4350,-13.0618,10.00,5.4,mww,NaN,51.0,2.797,0.79,...,2022-02-10T12:30:17.501Z,Ascension Island region,earthquake,10.3,1.9,0.098,10.0,reviewed,us,us
2,2022-02-10T09:47:19.772Z,-20.3017,-174.4717,46.12,4.9,mb,NaN,102.0,6.857,1.10,...,2022-02-10T10:13:21.040Z,"55 km SSW of Pangai, Tonga",earthquake,6.2,8.9,0.092,37.0,reviewed,us,us
3,2022-02-10T09:38:32.883Z,23.9793,122.2855,22.31,4.6,mb,NaN,58.0,0.660,0.67,...,2022-02-10T10:47:24.040Z,"69 km E of Hualien City, Taiwan",earthquake,2.1,5.4,0.100,30.0,reviewed,us,us
4,2022-02-10T04:40:52.721Z,-34.2923,179.5410,10.00,5.1,mb,NaN,114.0,3.413,1.02,...,2022-02-10T04:58:53.040Z,south of the Kermadec Islands,earthquake,6.3,1.9,0.089,43.0,reviewed,us,us


In [2]:
# hex color function categorizing points by 'magnitude'
def circle_color(mag):
    if 4.5 >= mag <= 5.4:
        return '#EDDE00' # yellow
    elif 5.5 >= mag <= 6.4:
        return '#FF8000' # orange
    elif mag >= 6.5:
        return '#E41316' # red

# size function
def circle_size(mag):
    if 4.5 >= mag <= 5.4:
        return mag*10000
    elif 5.5 >= mag <= 6.4:
        return mag*20000
    elif mag >= 6.5:
        return mag*60000

In [3]:
import folium

# display basemap
map_world = folium.Map(location=[0, 0], tiles='openstreetmap', zoom_start=2, crs='EPSG3857')

# add tectonic plate boundaries from GeoJSON file
url = 'https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json'
style = {'fillColor': 'purple',
         'color': 'purple',
         'dashArray': '10, 5',
         'weight': 2}
folium.GeoJson(url, name='Plate borders', style_function=lambda x: style).add_to(map_world)

# add markers
for row in data.itertuples():
    mag = getattr(row, 'mag')
    depth = getattr(row, 'depth')
    time = getattr(row, 'time')
    loc = getattr(row, 'place')
    tooltip = f"""Magnitude = {mag}<br>Depth = {depth} [km]<br>Time = {time}<br>Place = {loc}"""
    folium.Circle(location=[row.latitude, row.longitude], radius=circle_size(row.mag), fill_opacity=0.3,
                  tooltip=tooltip, color=circle_color(row.mag), fill=True, weight=1).add_to(map_world)

# adding layer control
folium.LayerControl('topright', collapsed=False).add_to(map_world)

# displaying in window mode
map_world